# Generalist-Specialist Score
This score measures the diversity of the sharing of a musician. 

In [1]:
import pandas as pd
import math 
from sklearn.metrics import pairwise 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly
import plotly.figure_factory as ff
from scipy import stats
import os
os.chdir(r'C:\Users\DELL XPS\Desktop\reddit_music')

metadata=pd.read_table('data/reddit-master-metadata.tsv')
vectors=pd.read_table("data/reddit-master-vectors.tsv",names=list(range(150)))
metadata=pd.DataFrame(metadata['community'])
genres = pd.read_csv("data\\genre_names_f.csv")
artist_vector = pd.read_csv("data\\Artist_Vectors_f.csv")
artist_counts = pd.read_csv('data\\artist_counts.csv')
data=pd.merge(metadata,vectors,left_index=True,right_index=True)
df = pd.read_csv('data\\Reddit_Sub_Com_f_genre.csv')

In [2]:
grouped = df.groupby(['artist','subreddit'])['score'].count()
grouped = grouped.reset_index()
grouped.columns = ['artist','subreddit','count']
grouped = pd.merge(grouped, data, left_on = 'subreddit',right_on = 'community')


dimensions = [k for k in range(150)]
dimensions1 = [str(k) for k in range(150)]

artist_vector.index = artist_vector['artist']
artist_vector = artist_vector[dimensions1]
artists = artist_vector.index

artist_vector_n = (artist_vector.T  / [abs(k) for k in artist_vector.sum(axis = 1).values]).T
artist_vector_n.to_csv('data\\Artist_Vector_n_v4.csv')

values = []
for i, a in enumerate(artists):
    print(str(i)+ ': ' + a)
    temp = grouped[grouped['artist']==a]
    subs = temp['subreddit'].unique()
    J = temp['count'].sum()
    artist_v = artist_vector_n.loc[artist_vector_n.index == a][dimensions1].values
    score = 0
    for sub in subs: 
        sub_df = temp[temp['subreddit']==sub]
        count = sub_df['count'].values[0]
        score += pairwise.cosine_similarity(sub_df[dimensions].values.reshape(1,-1), artist_v.reshape(1,-1)) * count
    score = score / J
    values.append({'artist': a,'GS_score':score[0][0]})


gs_score = pd.DataFrame(values).sort_values(by=  'GS_score')
gs_score.reset_index(inplace=True,drop=True)


0: "Da" Y
1: "Weird Al" Yankovic
2: #1 Funny Ringtones for Android Best New Annoying Comedy Parody Alerts Alarms Message Tones Tone Alert & Messages
3: $Ha Hef
4: $teven Cannon
5: $uicideBoy$
6: '68
7: (G)I-DLE
8: (Sandy) Alex G
9: *NSYNC
10: 009 Sound System
11: 03 Greedo
12: 070 Shake
13: 10 Years
14: 100%
15: 1000mods
16: 10cc
17: 11 Years of This
18: 12th Planet
19: 12xAM
20: 1788-L
21: 1991
22: 199X
23: 2 Chainz
24: 20syl
25: 21 Savage
26: 22
27: 257ers
28: 2CELLOS
29: 2NE1
30: 2PM
31: 2Pac
32: 2WEI
33: 2play4you
34: 3 Doors Down
35: 311
36: 3LAU
37: 3TEETH
38: 4 Non Blondes
39: 40 Watt Sun
40: 45 Grave
41: 4hero
42: 4th Lyric
43: 5 Seconds of Summer
44: 50 Cent
45: 6 Dogs
46: 65daysofstatic
47: 6LACK
48: 6ix9ine
49: 727's
50: 7DD9
51: 8 Bit Universe
52: 8-Bit Arcade
53: 8-Off Agallah Featuring Sean Price & Bazaar Royale
54: 88GLAM
55: 88rising
56: 99 Souls
57: 9MUSES
58: :Wumpscut:
59: A Boogie Wit da Hoodie
60: A Day To Remember
61: A Fine Frenzy
62: A Flock Of Seagulls
63: A Gr

KeyboardInterrupt: 

In [4]:
artist_song = pd.read_csv('data\\artist-song.csv')
artist_song = artist_song[['artist','track_name','proporition']]
artist_song.columns = ['artist','track_name','proportion']

gs_score = pd.merge(artist_song, gs_score, on ='artist',how='right')
gs_score.sort_values(by='GS_score', inplace=True)

gs_score = pd.merge(gs_score, artist_counts, on = 'artist',how='left')

gs_score.to_csv("data\\gs_score_f.csv", index=  False)

## Pecentile method
Here we normalize the GS score as outlined in paper. 

In [3]:
gs_score = pd.read_csv("data\\gs_score_f.csv")
gs_score['percentile'] = 0
dic = {'old_value': gs_score['count'].min()}
for i in np.arange(0.1, 1.1, .1):
    val = gs_score['count'].quantile(i)
    gs_score.loc[(gs_score['count'] <= val) & (gs_score['count'] >= dic['old_value']), 'percentile'] = int(i * 100)
    dic.update({'old_value': val})

lis = []
dic = {'old_value': gs_score['count'].min()}
for i in np.arange(0.1, 1.1, .1):
    val = gs_score['count'].quantile(i)
    lis.append({'percentile': int(i * 100), 'values': gs_score.loc[(gs_score['count'] <= val) & (gs_score['count'] >= dic['old_value']),'GS_score'].values})
    dic.update({'old_value': val})



values = [k['values'] for k in lis]
grouped_names = [k['percentile'] for k in lis]

fig = ff.create_distplot(values, grouped_names, show_hist=False, show_rug=False, )
fig.update_layout(
    xaxis_title = 'GS score',
    yaxis_title = 'Percentage of users',
    legend_title = 'Sharing percentile'
    )
fig.write_image('visuals\\image.png', engine = 'kaleido', scale = 3)


gs_score_80 = gs_score[gs_score['percentile'] == 80]
gs_score_90 = gs_score[gs_score['percentile'] == 90]
gs_score_100 = gs_score[gs_score['percentile'] == 100]

gs_score_100['percentile2'] = gs_score_100['GS_score'].apply(lambda x: stats.percentileofscore(gs_score_100['GS_score'].values, x))
gs_score_90['percentile2'] = gs_score_90['GS_score'].apply(lambda x: stats.percentileofscore(gs_score_90['GS_score'].values, x))
gs_score_80['percentile2'] = gs_score_80['GS_score'].apply(lambda x: stats.percentileofscore(gs_score_80['GS_score'].values, x))

gs_score_f = gs_score_80.append(gs_score_90.append(gs_score_100))
gs_score_f = gs_score_f.sort_values(by='percentile2')
gs_score_f.to_csv("data\\gs_score_percentile_f.csv")

In [3]:
gs_score= pd.read_csv("data\\gs_score_percentile_f.csv")
gs_score['count'].min()

124

In [9]:
min_score = gs_score_f['percentile2'].min()
max_score = gs_score_f['percentile2'].max()

diff = max_score - min_score

min_score2 = min_score/diff

gs_score_f['Normalized score'] = gs_score_f['percentile2'].apply(lambda x: -(x/diff) + min_score2 + 1)

In [12]:
gs_score_temp = gs_score_f.dropna(subset=['track_name'])
val = ""
for i, row in gs_score_temp[['artist','track_name','Normalized score']].tail(10).iterrows():
    val += "& {} & {} & ${}$\\\ ".format(str(row[0]).replace("&", "\&"),str(row[1]).replace("&", "\&"),round(row[2],3))

In [13]:
print(val)

& MC Rich & Down the Coast & $0.005$\\ & Lovelyz & Destiny & $0.003$\\ & Immortal Technique & Dance With the Devil & $0.003$\\ & WINNER & SENTIMENTAL & $0.003$\\ & Phinehas & White Livered & $0.002$\\ & Jay Rock & Vice City (feat. Black Hippy) & $0.002$\\ & Wvs & Cotton Candy & $0.002$\\ & dom champ & I Feel Better & $0.0$\\ & Invent, Animate & Darkbloom & $0.0$\\ & SHINee & View & $0.0$\\ 


,score,link_id,author,subreddit,created_utc,id,parent_id,track_id,artist,track_name,link,youtube_id,class,url,album_id,label
2878,4.0,t3_cy2zrj,i-yeet-3000,teenagers,1.567296e+09,t1_eypcdwm,t3_cy2zrj,5c6VfU899XdfvYFCLwUXxq,Rick Astley,Never Gonna Give You Up,youtube.com/watch?v=dQw4w9WgXcQ,youtube.com/watch?v=dQw4w9WgXcQ,comment,NaN,NaN,41
2879,1.0,t3_dpxpy6,imnotacreativechild,teenagers,1.572574e+09,t1_f5zdaic,t1_f5zd7qu,5c6VfU899XdfvYFCLwUXxq,Rick Astley,Never Gonna Give You Up,youtube.com/watch?v=dQw4w9WgXcQ,youtube.com/watch?v=dQw4w9WgXcQ,comment,NaN,NaN,41
2880,6.0,t3_dbla5q,Skyymaster7,teenagers,1.569896e+09,t1_f22smqt,t1_f22rrqm,5c6VfU899XdfvYFCLwUXxq,Rick Astley,Never Gonna Give You Up,youtube.com/watch?v=dQw4w9WgXcQ,youtube.com/watch?v=dQw4w9WgXcQ,comment,NaN,NaN,41
2881,3.0,t3_b7v16y,GenPrezEisenhower,teenagers,1.554079e+09,t1_ejud9gy,t3_b7v16y,5c6VfU899XdfvYFCLwUXxq,Rick Astley,Never Gonna Give You Up,youtube.com/watch?v=dQw4w9WgXcQ,youtube.com/watch?v=dQw4w9WgXcQ,comment,NaN,NaN,41
2882,2.0,t3_e42ff4,JussDoIt_,teenagers,1.575163e+09,t1_f97ap2k,t1_f977sqk,5c6VfU899XdfvYFCLwUXxq,Rick Astley,Never Gonna Give You Up,youtube.com/watch?v=dQw4w9WgXcQ,youtube.com/watch?v=dQw4w9WgXcQ,comment,NaN,NaN,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101173,2.0,t3_26wk7z,willr455,TelevisionQuotes,NaN,t1_cj78ae6,t1_chvhcjd,76QJgXPE3oc1cnkE4PFHEn,Rick Astley,Together Forever,youtube.com/watch?v=yPYZpwSpKmA,youtube.com/watch?v=yPYZpwSpKmA,comment,NaN,NaN,41
101174,5.0,t3_bwatky,MrZJones,Cityofheroes,1.559578e+09,t1_epwjqsr,t3_bwatky,76QJgXPE3oc1cnkE4PFHEn,Rick Astley,Together Forever,youtube.com/watch?v=yPYZpwSpKmA,youtube.com/watch?v=yPYZpwSpKmA,comment,NaN,NaN,41
101175,3.0,t3_jwqoi,tbwIII,webdesign,NaN,t1_c2fuv3d,t3_jwqoi,76QJgXPE3oc1cnkE4PFHEn,Rick Astley,Together Forever,youtube.com/watch?v=yPYZpwSpKmA,youtube.com/watch?v=yPYZpwSpKmA,comment,NaN,NaN,41
101176,5.0,t3_4oocpz,ahhter,Full_news,1.466287e+09,t1_d4epkhh,t3_4oocpz,6TdHgARbMSVuftE4Lb0dHY,Rick Astley,Keep Singing,youtube.com/watch?v=AC3Ejf7vPEY,youtube.com/watch?v=AC3Ejf7vPEY,comment,NaN,NaN,41
